# Sistem Identifikasi Suara

In [ ]:
project/
│
├── data/                           # Folder data suara mentah
│   ├── user1/
│   │   ├── buka/                   # Contoh perintah "buka" dari user1
│   │   └── tutup/                  # Contoh perintah "tutup" dari user1
│   ├── user2/
│   └── voice_dataset.csv           # Metadata hasil ekstraksi fitur suara
│
├── models/                         # Model hasil pelatihan (tersimpan .pkl)
│   ├── feature_cols.pkl            # Kolom fitur yang digunakan saat training
│   ├── status_model.pkl            # Model klasifikasi status (buka/tutup)
│   └── user_model.pkl              # Model klasifikasi pengguna (user1/user2)
│
├── utils/
│   ├── __init__.py                 # Penanda folder Python package
│   ├── feature_extraction.py       # Fungsi ekstraksi fitur audio (MFCC, Zero-Cross, dll.)
│   └── feature_extraction.py              # 🔥 Fungsi utama: training, evaluasi, dan prediksi model
│
├── app.py                          # Aplikasi utama (misal Streamlit / Flask)
├── generate_dataset.py             # Membuat CSV dataset dari folder suara mentah
├── train_model.py                  # Melatih model utama (status + user)
├── train_status.py                 # Melatih model pengenalan status buka/tutup
├── train_user.py                   # Melatih model pengenalan user (speaker)
├── requirements.txt                # Daftar library Python yang dibutuhkan
└── README.md                       # Dokumentasi proyek


##  File: `requirements.txt`
yaitu daftar dependensi (library) yang dibutuhkan agar aplikasi bisa berjalan di server atau saat deployment

In [ ]:
streamlit==1.37.0
streamlit-audiorec==0.1.3
librosa==0.10.1
numpy==1.26.4
pandas==2.2.2
scikit-learn==1.4.2
joblib==1.4.2
soundfile==0.12.1
audioread==3.0.1
numba==0.59.1
matplotlib==3.9.0
SpeechRecognition==3.11.0
llvmlite==0.42.0

**Fungsi Tiap Library**
| Library                       | Versi                     | Fungsi Utama                                                                                               | Digunakan di File                                           |
| ----------------------------- | ------------------------- | ---------------------------------------------------------------------------------------------------------- | ----------------------------------------------------------- |
| **streamlit==1.37.0**         | UI framework              | Untuk membuat web app interaktif seperti `app.py` (menampilkan input, tombol, grafik, dan hasil prediksi). | `app.py`                                                    |
| **streamlit-audiorec==0.1.3** | Audio recorder widget     | Untuk **merekam suara langsung dari browser** dan memasukkannya ke aplikasi Streamlit.                     | `app.py` (fitur perekaman suara pengguna)                   |
| **librosa==0.10.1**           | Audio processing          | Untuk **ekstraksi fitur suara (MFCC, Chroma, ZCR)** dari file .wav atau hasil rekaman.                     | `utils/feature_extraction.py`                               |
| **numpy==1.26.4**             | Komputasi numerik         | Untuk operasi matematis cepat (array, mean, std, reshape).                                                 | Semua file model dan ekstraksi                              |
| **pandas==2.2.2**             | Pengolahan data tabular   | Untuk membaca dataset (`voice_dataset.csv` / `NO2_Pademawu.csv`), membuat DataFrame hasil prediksi.        | `model_utils.py`, `train_*.py`, `app.py`                    |
| **scikit-learn==1.4.2**       | Machine learning          | Untuk membuat dan melatih model seperti **RandomForestClassifier / Regressor**, serta evaluasi MAPE.       | `model_utils.py`, `train_user.py`, `train_status.py`        |
| **joblib==1.4.2**             | Model serialization       | Untuk **menyimpan dan memuat model** (`.pkl` files) agar tidak perlu retrain setiap kali.                  | `model_utils.py`, `train_*.py`                              |
| **soundfile==0.12.1**         | Audio I/O                 | Untuk **membaca dan menulis file audio** (format WAV/FLAC).                                                | `feature_extraction.py`                                     |
| **audioread==3.0.1**          | Audio backend             | Backend alternatif bagi `librosa` untuk membaca file audio di berbagai format (mp3, wav, flac).            | `feature_extraction.py`                                     |
| **numba==0.59.1**             | JIT compiler untuk Python | Digunakan otomatis oleh `librosa` agar ekstraksi fitur suara lebih cepat.                                  | Tidak langsung kamu panggil, tapi digunakan oleh `librosa`. |
| **matplotlib==3.9.0**         | Visualisasi grafik        | Untuk membuat **grafik prediksi NO₂** dan data historis di `app.py`.                                       | `app.py`                                                    |
| **SpeechRecognition==3.11.0** | Speech-to-text engine     | Untuk **mengubah hasil suara menjadi teks** (contohnya mengenali kata “buka” atau “tutup”).                | `train_status.py`, `app.py` (jika ada deteksi perintah)     |
| **llvmlite==0.42.0**          | Dependensi Numba          | Paket internal yang dibutuhkan oleh `numba` agar dapat melakukan kompilasi cepat.                          | Digunakan otomatis                                          |


##  File: `generate_dataset.py`
fungsinya sangat penting dalam seluruh proyekmu karena inilah tahap pembentukan dataset utama dari file suara sebelum model dilatih.


In [ ]:
import os
import pandas as pd
from utils.feature_extraction import extract_features

# === Folder data utama ===
DATA_DIR = "data"

data = []

# Loop semua user (user1, user2, dst)
for user_name in os.listdir(DATA_DIR):
    user_path = os.path.join(DATA_DIR, user_name)
    if not os.path.isdir(user_path):
        continue

    # Loop subfolder "buka" dan "tutup"
    for status_name in ["buka", "tutup"]:
        status_path = os.path.join(user_path, status_name)
        if not os.path.isdir(status_path):
            print(f"[PERINGATAN] Folder {status_path} tidak ditemukan, dilewati.")
            continue

        # Loop semua file .wav di dalam folder
        for file_name in os.listdir(status_path):
            if file_name.lower().endswith(".wav"):
                file_path = os.path.join(status_path, file_name)
                feats = extract_features(file_path)

                if feats:
                    feats["user"] = user_name      # contoh: user1 / user2
                    feats["status"] = status_name  # contoh: buka / tutup
                    data.append(feats)

# Simpan dataset
if data:
    df = pd.DataFrame(data)
    os.makedirs("data", exist_ok=True)
    output_path = os.path.join("data", "voice_dataset.csv")
    df.to_csv(output_path, index=False)
    print(f"[INFO] Dataset tersimpan: {len(df)} sampel suara → {output_path}")
else:
    print("[PERINGATAN] Tidak ada data suara ditemukan.")


**Fungsi Utama generate_dataset.py**
- Membaca semua file .wav di dalamnya
  - Setiap file diambil jalurnya penuh (file_path).
- Menjalankan fungsi extract_features(file_path)
  - Mengambil ciri suara (fitur) seperti MFCC
  - Hasilnya berupa dictionary fitur untuk tiap file.
- Menambahkan label user dan status
  - user = user1  
  - status = buka
- Menggabungkan semua data ke DataFrame dan menyimpan ke CSV
  - File hasil: data/voice_dataset.csv.

##  File: `extract_features.py`
Menarik ciri khas (fitur numerik) dari file suara .wav menggunakan MFCC agar bisa digunakan untuk pelatihan model machine learning. feature_extraction.py adalah **tahap preprocessing** audio mentah menjadi data numerik (fitur MFCC).

In [ ]:
import numpy as np
import librosa

# Jumlah koefisien MFCC yang diharapkan model (36 fitur)
N_MFCC_EXPECTED = 36

def extract_features(file_path):
    """
    Ekstraksi fitur MFCC (36 koefisien) dari file WAV.
    Mengembalikan dictionary fitur dengan keys 'mfcc0' hingga 'mfcc35'.
    """

    try:
        # Load audio file
        y, sr = librosa.load(file_path, sr=None)
        # Normalize the signal
        y = librosa.util.normalize(y)

        # Ekstraksi dan rata-rata MFCCs (36 koefisien)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=N_MFCC_EXPECTED)
        mfcc_feats = np.mean(mfccs.T, axis=0)

        feature_dict = {}

        # Penamaan kolom: mfcc0, mfcc1, ..., mfcc35
        if len(mfcc_feats) == N_MFCC_EXPECTED:
            for i, val in enumerate(mfcc_feats):
                feature_dict[f"mfcc{i}"] = float(val)
        else:
            # Pengecekan error jika jumlah fitur yang diekstrak salah
            print(f"[ERROR] Jumlah MFCC yang diekstrak salah: {len(mfcc_feats)}. Diharapkan: {N_MFCC_EXPECTED}")
            return None


        # Mengembalikan dictionary fitur
        return {k: np.nan_to_num(v) for k, v in feature_dict.items()}

    except Exception as e:
        print(f"[ERROR] Gagal memproses {file_path}: {e}")
        return None

**Fungsi utama extract_features.py**
- Dipanggil oleh generate_dataset.py untuk setiap file .wav.

- Hasilnya berupa fitur numerik (mfcc0–mfcc35) yang disimpan dalam dataset CSV.

- Dataset ini nanti dipakai untuk melatih model klasifikasi atau prediksi (misalnya deteksi suara “buka” vs “tutup”).

##  File: `train_model.py`
Melatih dua model machine learning (Logistic Regression) untuk mengenali pengguna (user) dan status suara (“buka” / “tutup”) berdasarkan fitur MFCC dari file suara.

In [ ]:
import os
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from utils.feature_extraction import extract_features
# CATATAN: Pastikan Anda sudah membuat folder 'utils' dan meletakkan feature_extraction.py di dalamnya.

# ========================= KONFIGURASI PATH DATA =========================
# Ganti dengan path folder tempat data Anda disimpan
DATA_ROOT = 'data'

# Struktur data yang Diharapkan:
# DATA_ROOT/user1/buka/*.wav
# DATA_ROOT/user1/tutup/*.wav
# DATA_ROOT/user2/buka/*.wav
# DATA_ROOT/user2/tutup/*.wav

# ========================= 1. FUNGSI EKSTRAKSI DATA =========================

def load_and_extract_data(data_root):
    """Loads all WAV files and extracts features for both User and Status classification."""
    all_features = []

    for user_id in ['user1', 'user2']:
        for status_id in ['buka', 'tutup']:
            folder_path = os.path.join(data_root, user_id, status_id)
            if not os.path.exists(folder_path):
                print(f"WARNING: Folder tidak ditemukan: {folder_path}. Melewatkan.")
                continue

            for filename in os.listdir(folder_path):
                if filename.endswith('.wav'):
                    file_path = os.path.join(folder_path, filename)
                    features = extract_features(file_path)

                    if features:
                        # Tambahkan label (Target) ke dictionary fitur
                        features['target_user'] = 1 if user_id == 'user1' else 2
                        features['target_status'] = 0 if status_id == 'buka' else 1
                        all_features.append(features)

    # Buat DataFrame dari semua data
    df = pd.DataFrame(all_features)
    print(f"\nTotal {len(df)} sampel data berhasil diekstrak.")
    return df

# ========================= 2. PROSES PELATIHAN =========================

def train_and_save_models(df):
    if df.empty:
        print("ERROR: Tidak ada data untuk dilatih.")
        return

    # Menentukan fitur (X) dan label (y)
    feature_cols = [col for col in df.columns if col not in ['target_user', 'target_status']]
    X = df[feature_cols]

    # Label untuk User Identification (y_user) dan Status (y_status)
    y_user = df['target_user']
    y_status = df['target_status']

    # --- Persiapan Pelatihan ---
    if not os.path.exists('models'):
        os.makedirs('models')

    # Simpan daftar fitur (Penting untuk app.py)
    joblib.dump(feature_cols, 'models/feature_cols.pkl')
    print(f"Daftar fitur disimpan ke models/feature_cols.pkl (Total: {len(feature_cols)} fitur)")


    # --- Pelatihan Model User (user_model.pkl) ---
    print("\nMelatih Model User Identification...")
    X_train_u, X_test_u, y_train_u, y_test_u = train_test_split(X, y_user, test_size=0.2, random_state=42)

    # Menggunakan Logistic Regression atau model sederhana lainnya
    user_model = LogisticRegression(max_iter=500)
    user_model.fit(X_train_u, y_train_u)

    # Evaluasi
    y_pred_u = user_model.predict(X_test_u)
    accuracy_u = accuracy_score(y_test_u, y_pred_u)
    print(f"Akurasi Model User di data test: {accuracy_u * 100:.2f}%")

    joblib.dump(user_model, 'models/user_model.pkl')
    print("Model User disimpan ke models/user_model.pkl")

    # --- Pelatihan Model Status (status_model.pkl) ---
    print("\nMelatih Model Status (Buka/Tutup)...")
    X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X, y_status, test_size=0.2, random_state=42)

    status_model = LogisticRegression(max_iter=500)
    status_model.fit(X_train_s, y_train_s)

    # Evaluasi
    y_pred_s = status_model.predict(X_test_s)
    accuracy_s = accuracy_score(y_test_s, y_pred_s)
    print(f"Akurasi Model Status di data test: {accuracy_s * 100:.2f}%")

    joblib.dump(status_model, 'models/status_model.pkl')
    print("Model Status disimpan ke models/status_model.pkl")

    print("\n*** PELATIHAN SELESAI. SILAKAN JALANKAN app.py LAGI. ***")


# ========================= 3. MAIN EXECUTION =========================

if __name__ == '__main__':
    # 1. Muat dan Ekstrak Data
    data_df = load_and_extract_data(DATA_ROOT)

    # 2. Latih dan Simpan Model
    if not data_df.empty:
        train_and_save_models(data_df)

**Langkah utamanya:**
- Membaca semua file suara dari folder data/.

- Menjalankan ekstraksi fitur (menggunakan extract_features).

- Membuat DataFrame yang berisi fitur + label user/status.

- Melatih dua model:

  - user_model.pkl → mengenali siapa penggunanya.

  - status_model.pkl → mengenali apakah suara “buka” atau “tutup”.

- Menyimpan model dan metadata fitur ke folder models/ agar bisa dipakai di app.py.

##  File: `train_status.py`
Melatih model klasifikasi Random Forest untuk membedakan suara “buka” dan “tutup” berdasarkan fitur MFCC dari dataset voice_dataset.csv.
Model yang sudah jadi disimpan sebagai models/status_model.pkl agar bisa digunakan di aplikasi utama (app.py).

In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import joblib

# === Load dataset ===
df = pd.read_csv("data/voice_dataset.csv")

# === Fitur 36 kolom ===
feature_cols = [col for col in df.columns if col.startswith("mfcc")]
X = df[feature_cols].to_numpy()
y = df["status"].to_numpy()  # 0=buka, 1=tutup

# === Split data ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# === Pipeline RandomForest + StandardScaler ===
model_status = Pipeline([
    ("scaler", StandardScaler()),
    ("rf", RandomForestClassifier(n_estimators=200, random_state=42))
])

# === Training ===
model_status.fit(X_train, y_train)

# === Simpan model dan feature order ===
os.makedirs("models", exist_ok=True)
joblib.dump(model_status, "models/status_model.pkl")
joblib.dump(feature_cols, "models/feature_cols.pkl")
print("[INFO] Model status berhasil disimpan.")


**Fungsi Utama train_status.py**
- melatih dan menyimpan model status buka/tutup suara dengan algoritma Random Forest, berbasis pada fitur MFCC (Mel-Frequency Cepstral Coefficients)
- Memisahkan data jadi data latih (80%) dan data uji (20%) agar model bisa dievaluasi dengan adil.
- Menggabungkan dua langkah otomatis:
  - StandardScaler untuk menormalkan fitur
  - RandomForestClassifier sebagai algoritma pelatihan
- Menjalankan proses belajar dari data latih untuk mengenali pola suara buka/tutup.

##  File: `train_user.py`
Melatih model machine learning untuk mengenali siapa pengguna (user1 atau user2) berdasarkan ciri suara (fitur MFCC), lalu menyimpannya dalam bentuk file model .pkl agar bisa digunakan di aplikasi utama (app.py).

In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import joblib

# === Load dataset ===
df = pd.read_csv("data/voice_dataset.csv")

# === Cek kolom ===
if "user" not in df.columns:
    raise ValueError("Kolom 'user' tidak ditemukan di voice_dataset.csv. Jalankan generate_dataset.py dulu.")

# === Fitur ===
feature_cols = [col for col in df.columns if col.startswith("mfcc")]
X = df[feature_cols].to_numpy()
y = df["user"].to_numpy()

# === Split data ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# === Pipeline RandomForest + StandardScaler ===
model_user = Pipeline([
    ("scaler", StandardScaler()),
    ("rf", RandomForestClassifier(n_estimators=200, random_state=42))
])

# === Training ===
model_user.fit(X_train, y_train)

# === Simpan model dan fitur ===
os.makedirs("models", exist_ok=True)
joblib.dump(model_user, "models/user_model.pkl")
joblib.dump(feature_cols, "models/feature_cols.pkl")

print("[INFO] Model user berhasil disimpan.")


**Fungsi Utama train_user.py**
- Memuat data hasil ekstraksi fitur suara dari voice_dataset.csv.
- Mengambil kolom mfcc0-mfcc35 yang berisi 36 nilai fitur akustik dari setiap file suara.
- melatih dan menyimpan model identifikasi pengguna suara (user recognition) menggunakan algoritma Random Forest berbasis pada fitur MFCC hasil ekstraksi dari file audio.
- mengajarkan komputer mengenali siapa yang berbicara (user1 atau user2) dari pola ciri suara mereka

##  File: `app.py`
yaitu aplikasi Streamlit yang menjalankan sistem identifikasi suara buka/tutup dan verifikasi pengguna (user1/user2).

In [ ]:
import streamlit as st
import numpy as np
import joblib
import tempfile
import pandas as pd
from utils.feature_extraction import extract_features
from st_audiorec import st_audiorec

# ===== KONSTANTA: AMBANG BATAS KEYAKINAN =====
# Keyakinan minimum yang dibutuhkan model untuk mengidentifikasi pengguna (70%)
CONFIDENCE_THRESHOLD = 0.70

# ===== CONFIG =====
st.set_page_config(page_title="Voice Identification", page_icon="🎧", layout="centered")
st.title("🎙️ Sistem Identifikasi Suara")
st.markdown(f"""
Sistem ini melakukan identifikasi suara **buka/tutup** dari dua pengguna yang diizinkan:
**user1** dan **user2**.
Jika keyakinan model di bawah **{CONFIDENCE_THRESHOLD*100:.0f}%**, pengguna akan ditandai **TIDAK DIKENALI**.
""")

# ===== LOAD MODEL =====
try:
    # Load user classification model
    model_user = joblib.load("models/user_model.pkl")
    # Load status classification model (buka/tutup)
    model_status = joblib.load("models/status_model.pkl")
    # Load the list of feature column names
    feature_cols = joblib.load("models/feature_cols.pkl")

except FileNotFoundError:
    st.error("Gagal memuat file model. Pastikan 'user_model.pkl', 'status_model.pkl', dan 'feature_cols.pkl' ada di folder 'models/'.")
    st.stop()
except Exception as e:
    st.error(f"Gagal memuat model: {e}")
    st.stop()

# ===== PILIH INPUT =====
input_option = st.radio("Pilih metode input:", ["🎤 Rekam suara", "📁 Upload file .wav"])

# ===== PROSES AUDIO =====
def process_audio(audio_path):
    import librosa
    import speech_recognition as sr

    # Extract features using the utility function
    features = extract_features(audio_path)
    if not features:
        st.error("Gagal mengekstraksi fitur dari suara.")
        return

    # ===== DETEKSI SILENCE =====
    try:
        y, sr_audio = librosa.load(audio_path, sr=None)
    except Exception as e:
        st.error(f"Gagal memuat file audio: {e}")
        return

    rms = np.mean(librosa.feature.rms(y=y))
    duration = len(y)/sr_audio

    # Check minimum duration and volume
    if duration < 0.1 or rms < 1e-4:
        st.error("Tidak ada suara yang terdeteksi. Silakan rekam ulang.")
        return

    # Convert features to DataFrame and then to the model input array
    feature_df = pd.DataFrame([features])

    # Ensure column order matches training data
    try:
        X = feature_df[feature_cols].to_numpy().reshape(1, -1)
    except KeyError as e:
        st.error(f"Kolom fitur tidak cocok dengan model yang dilatih. Pastikan 'feature_extraction.py' sudah benar. Error: {e}")
        st.write("Kolom yang Diharapkan:", feature_cols)
        st.write("Kolom yang Ditemukan:", feature_df.columns.tolist())
        return

    # TIDAK ADA SCALING DILAKUKAN
    st.write(f"Jumlah fitur yang dikirim ke model: {X.shape[1]} (tanpa scaling)")


    # ===== PREDIKSI USER & STATUS =====
    # Prediksi menggunakan data X (tanpa scaling)
    user_pred_proba = model_user.predict_proba(X)
    status_pred_proba = model_status.predict_proba(X)

    user_pred = np.argmax(user_pred_proba)
    status_pred = np.argmax(status_pred_proba)

    # Confidence
    user_confidence = np.max(user_pred_proba)
    status_confidence = np.max(status_pred_proba)

    # --- IMPLEMENTASI THRESHOLD ---
    is_identified = False
    if user_confidence >= CONFIDENCE_THRESHOLD:
        user_label = f"user{user_pred + 1}"
        status_label = "buka" if status_pred == 0 else "tutup"
        is_identified = True
    else:
        user_label = "TIDAK DIKENALI"
        status_label = "N/A"

    # ===== SPEECH TO TEXT (STT) =====
    recognizer = sr.Recognizer()
    stt_text = None
    try:
        with sr.AudioFile(audio_path) as source:
            audio_data = recognizer.record(source)
            stt_text = recognizer.recognize_google(audio_data, language="id-ID")
            st.subheader("📝 Teks yang terdeteksi")
            st.code(stt_text, language='text')

            # Koreksi prediksi status berdasarkan teks HANYA JIKA USER DIKENALI
            if is_identified and stt_text:
                 text_lower = stt_text.lower()
                 if "buka" in text_lower:
                     status_label = "buka"
                 elif "tutup" in text_lower:
                     status_label = "tutup"

    except Exception as e:
        st.warning(f"Teks tidak dapat dikenali: {e}")

    # ===== Hasil & Debugging Confidence =====
    st.subheader("📊 Hasil Prediksi")

    if is_identified:
        # Tampilkan hasil yang diizinkan
        st.success(f"✅ **Diizinkan:** {user_label} — **Aksi:** {status_label}")
        if status_label == "buka":
            st.write(f"🔊 Suara mirip **{user_label} saat membuka** sesuatu.")
        else:
            st.write(f"🔊 Suara mirip **{user_label} saat menutup** sesuatu.")
    else:
        # Tampilkan hasil yang ditolak
        st.error(f"❌ **TIDAK DIKENALI / Otorisasi Ditolak.**")
        st.warning(f"Suara tidak mencapai tingkat keyakinan minimum ({CONFIDENCE_THRESHOLD*100:.0f}%) untuk User 1 atau User 2.")

    # Tampilkan Confidence Score
    st.markdown("---")
    st.markdown(f"**Detail Keyakinan Model User (Threshold {CONFIDENCE_THRESHOLD*100:.0f}%):**")
    # Tampilkan probabilitas untuk kedua user
    st.info(f"Probabilitas User 1: **{user_pred_proba[0][0]*100:.2f}%** | Probabilitas User 2: **{user_pred_proba[0][1]*100:.2f}%**")
    st.markdown(f"**Keyakinan Status Buka/Tutup (Model):**")
    st.info(f"Keyakinan Status: **{status_confidence*100:.2f}%**")
    st.markdown("---")

    # ===== View extracted features =====
    with st.expander("📈 Lihat fitur yang diekstraksi"):
        st.dataframe(feature_df.T, use_container_width=True)

# ===== INPUT RECORDING =====
if input_option == "🎤 Rekam suara":
    st.write("Klik tombol di bawah untuk merekam suara Anda:")
    audio_bytes = st_audiorec()
    if audio_bytes is not None:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmpfile:
            tmpfile.write(audio_bytes)
            audio_path = tmpfile.name
        st.audio(audio_path, format="audio/wav")
        process_audio(audio_path)

# ===== INPUT UPLOAD =====
elif input_option == "📁 Upload file .wav":
    uploaded_file = st.file_uploader("Upload file suara (.wav):", type=["wav"])
    if uploaded_file is not None:
        st.audio(uploaded_file, format="audio/wav")
        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmpfile:
            tmpfile.write(uploaded_file.read())
        audio_path = tmpfile.name
        process_audio(audio_path)

**Fungsi Utama app.py**
- Mengambil input suara (rekaman/upload)
- Mengekstraksi fitur MFCC
- Melakukan prediksi user & status
- Menampilkan hasil identifikasi & confidence dengan antarmuka interaktif